In [71]:
import pandas as pd
import numpy as np
import itertools
import copy
import requests
import datetime
from datetime import date
import operator
import functools
import math
import time
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from espn_api.basketball import League

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
display(HTML("<style>.container { width:90% !important; }</style>"))

C:\Users\Jack\AppData\Local\Temp\ipykernel_58024\609247326.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [127]:
acronym_to_team_dict = {'ATL': 'Atlanta Hawks',
                        'BOS': 'Boston Celtics',
                        'BKN': 'Brooklyn Nets',
                        'CHA': 'Charlotte Hornets',
                        'CHI': 'Chicago Bulls',
                        'CLE': 'Cleveland Cavaliers',
                        'DAL': 'Dallas Mavericks',
                        'DEN': 'Denver Nuggets',
                        'DET': 'Detroit Pistons',
                        'GSW': 'Golden State Warriors',
                        'HOU': 'Houston Rockets',
                        'IND': 'Indiana Pacers',
                        'LAC': 'Los Angeles Clippers',
                        'LAL': 'Los Angeles Lakers',
                        'MEM': 'Memphis Grizzlies',
                        'MIA': 'Miami Heat',
                        'MIL': 'Milwaukee Bucks',
                        'MIN': 'Minnesota Timberwolves',
                        'NOP': 'New Orleans Pelicans',
                        'NYK': 'New York Knicks',
                        'OKC': 'Oklahoma City Thunder',
                        'ORL': 'Orlando Magic',
                        'PHL': 'Philadelphia 76ers',
                        'PHO': 'Phoenix Suns',
                        'POR': 'Portland Trail Blazers',
                        'SAC': 'Sacramento Kings',
                        'SAS': 'San Antonio Spurs',
                        'TOR': 'Toronto Raptors',
                        'UTA': 'Utah Jazz',
                        'WAS': 'Washignton Wizards'
                        }

team_to_acronym_dict = {'Atlanta Hawks': 'ATL',
                        'Boston Celtics': 'BOS',
                        'Brooklyn Nets': 'BKN',
                        'Charlotte Hornets': 'CHA',
                        'Chicago Bulls': 'CHI',
                        'Cleveland Cavaliers': 'CLE',
                        'Dallas Mavericks': 'DAL',
                        'Denver Nuggets': 'DEN',
                        'Detroit Pistons': 'DET',
                        'Golden State Warriors': 'GSW',
                        'Houston Rockets': 'HOU',
                        'Indiana Pacers': 'IND',
                        'Los Angeles Clippers': 'LAC',
                        'Los Angeles Lakers': 'LAL',
                        'Memphis Grizzlies': 'MEM',
                        'Miami Heat': 'MIA',
                        'Milwaukee Bucks': 'MIL',
                        'Minnesota Timberwolves': 'MIN',
                        'New Orleans Pelicans': 'NOP',
                        'New York Knicks': 'NYK',
                        'Oklahoma City Thunder': 'OKC',
                        'Orlando Magic': 'ORL',
                        'Philadelphia 76ers': 'PHL',
                        'Phoenix Suns': 'PHO',
                        'Portland Trail Blazers': 'POR',
                        'Sacramento Kings': 'SAC',
                        'San Antonio Spurs': 'SAS',
                        'Toronto Raptors': 'TOR',
                        'Utah Jazz': 'UTA',
                        'Washington Wizards': 'WAS'
                       }

def date_converter(d):
    month_day = d.split(',')[1]
    month = month_day.split(' ')[1]
    if month == 'Oct':
        month_num = '10'
    elif month == 'Nov':
        month_num = '11'
    elif month == 'Dec':
        month_num = '12'
    elif month == 'Jan':
        month_num = '01'
    elif month == 'Feb':
        month_num = '02'
    elif month == 'Mar':
        month_num = '03'
    elif month == 'Apr':
        month_num = '04'
    else:
        month_num = '00'
        
    day_num = str(month_day.split(' ')[2]).zfill(2)   # Lpadding with a 0
    year = str(d.split(', ')[2])[-2:]   # Shorteing to two numbers
    concat_date = month_num+'/'+day_num+'/'+year
    #new_date = datetime.datetime.strptime(concat_date, '%m/%d/%y').date()
    return concat_date

def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def date_to_week(d):
    week_num = 0
    
    if d <= datetime.datetime(2023, 10, 29):
        week_num = 1
    elif d <= datetime.datetime(2023, 11, 5):
        week_num = 2
    elif d <= datetime.datetime(2023, 11, 12):
        week_num = 3
    elif d <= datetime.datetime(2023, 11, 19):
        week_num = 4
    elif d <= datetime.datetime(2023, 11, 26):
        week_num = 5
    elif d <= datetime.datetime(2023, 12, 3):
        week_num = 6
    elif d <= datetime.datetime(2023, 12, 10):
        week_num = 7
    elif d <= datetime.datetime(2023, 12, 17):
        week_num = 8
    elif d <= datetime.datetime(2023, 12, 24):
        week_num = 9
    elif d <= datetime.datetime(2023, 12, 31):
        week_num = 10
    elif d <= datetime.datetime(2024, 1, 7):
        week_num = 11
    elif d <= datetime.datetime(2024, 1, 14):
        week_num = 12
    elif d <= datetime.datetime(2024, 1, 21):
        week_num = 13
    elif d <= datetime.datetime(2024, 1, 28):
        week_num = 14
    elif d <= datetime.datetime(2024, 2, 4):
        week_num = 15
    elif d <= datetime.datetime(2024, 2, 11):
        week_num = 16
    elif d <= datetime.datetime(2024, 3, 3):
        week_num = 17
    elif d <= datetime.datetime(2024, 3, 17):
        week_num = 18
    elif d <= datetime.datetime(2024, 3, 31):
        week_num = 19
    else:
        week_num = 0
        
    return week_num
        
def schedule_maker_23_24():
    column_names = ['Date', 'Week', 'Start_Time', 'Visitor', 'Visitor_PTS', 'Home', 'Home_PTS', 'Attendance', 'Arena', 'Notes']
    schedule_df = pd.DataFrame(columns=column_names)
    header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
    headers = {'User-Agent': header_name}

    url_start = 'https://www.basketball-reference.com/leagues/NBA_2024_games-'
    url_end = '.html'
    url_month_list = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
    url_list = []

    for month in url_month_list:
        url = url_start+month+url_end

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        table_container = soup.find('div', attrs={'id': 'div_schedule'})
        table = table_container.find('table', attrs={'id': 'schedule'})
        table_name_container = table.find('thead')

        #column_names = []

        #for cn in table_name_container.findAll('th'):
        #    column_names.append(cn.get_text().strip())

        table_body = table.find('tbody')
        for row in table_body.findAll('tr'):
            game_date = row.find('th', attrs={'data-stat': 'date_game'}).get_text().strip()
            if game_date == 'Date':   # Basketball Reference has intermediate headers with the column names and we don't want to include these
                continue
            else:  # Formatting date to MM/DD/YY
                game_date = date_converter(game_date)
            week_num = date_to_week(datetime.datetime.strptime(game_date, '%m/%d/%y'))
            game_time = row.find('td', attrs={'data-stat': 'game_start_time'}).get_text().strip()
            visitor = row.find('td', attrs={'data-stat': 'visitor_team_name'}).get_text().strip()
            visitor_pts = row.find('td', attrs={'data-stat': 'visitor_pts'}).get_text().strip()
            home = row.find('td', attrs={'data-stat': 'home_team_name'}).get_text().strip()
            home_pts = row.find('td', attrs={'data-stat': 'home_pts'}).get_text().strip()
            attendance = row.find('td', attrs={'data-stat': 'attendance'}).get_text().strip()
            arena = row.find('td', attrs={'data-stat': 'arena_name'}).get_text().strip()
            notes = row.find('td', attrs={'data-stat': 'game_remarks'}).get_text().strip()
            schedule_df = pd.concat([schedule_df, pd.DataFrame([[game_date, week_num, game_time, visitor, visitor_pts, home, home_pts, attendance, arena, notes]], columns=column_names)], axis=0, ignore_index=True)

        time.sleep(2)
 
    writing_data(schedule_df, filename='C:\\Users\\Jack\\Basketball Project\\NBA_Schedule_23_24.csv')   

In [128]:
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))

def week_day_teams(schedule_df):   
    week_day_df = {}
    for index, row in schedule_df.iterrows():

        if 'Week ' + str(row['Week']) not in week_day_df.keys():
            week_day_df['Week ' + str(row['Week'])] = {}

        if row['Date'] not in week_day_df['Week ' + str(row['Week'])].keys():
            week_day_df['Week ' + str(row['Week'])][row['Date']] = []

        week_day_df['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Visitor']])
        week_day_df['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Home']])

    return week_day_df 

def daily_players(week_num, player_dict, week_day_df):
    week = 'Week ' + str(week_num)
    weekly_dict = week_day_df[week]
    day_dict = {}

    for date, teams in weekly_dict.items():
        for team in teams:
            if team not in player_dict.keys():
                continue

            for player in player_dict[team]:
                if player.injuryStatus == 'OUT':
                    continue
                    
                if date not in day_dict.keys():
                    day_dict[date] = []
                    
                day_dict[date].append(player)

    return day_dict

def team_player_dict_creator(roster):
    player_team_dict = {}
    team_player_dict = {}

    for player in roster:
        player_team_dict[player] = player.proTeam

    for player, team in player_team_dict.items():
        if team not in team_player_dict.keys():
            team_player_dict[team] = [player]
        else:
            team_player_dict[team].append(player)
   
    return(team_player_dict)

def week_projected_points(week_num, week_day_df, team_player_dict):
    position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']

    day_dict = daily_players(week_num, team_player_dict, week_day_df) 
    day_projected_points = {}

    for day, players in day_dict.items():
        #print(list(itertools.permutations(players)))
        max_players = len(players)
        current_num_players = 0
        roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
        players_list = []
        total_projected_points = 0
        failed = 0
        combination_num = 0
        players_only_position = 0
        max_points = 0

        player_point_dict = {}
        for player in players:
            player_point_dict[player] = (player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2

        player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
        players_ordered_list = list(player_point_dict.keys())

        position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
        for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            for player in players_ordered_list:
                if position in player.eligibleSlots:
                    position_player_dict[position].append(player)

        position_only_players = []  # players who are the only player at a position
        one_player_positions = []
        no_one_positon = False  # we will loop again if we have a success
        empty_positions = []  # no players at position
        while no_one_positon == False:

            no_one_positon = True
            for position, players in position_player_dict.items():
                if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                    empty_positions.append(position) 

                if len(players) == 1 and players[0] not in position_only_players:
                    one_player_positions.append(position)
                    roster_spots[position] = players[0]
                    total_projected_points+=(players[0].avg_points+(players[0].avg_points if players[0].projected_avg_points==0 else players[0].projected_avg_points))/2
                    players_ordered_list.remove(players[0])
                    position_only_players.append(players[0])
                    players_only_position+=1
                    no_one_positon = False

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1])
                            #position_player_dict[position2].remove(position_only_players[-1])

                if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                    or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                    if roster_spots['PG'] == None:
                        roster_spots['PG'] = position_player_dict['PG'][0]
                        roster_spots['G'] = position_player_dict['PG'][1]  
                        one_player_positions.append('PG')
                        one_player_positions.append('G')                            
                        total_projected_points+=(position_player_dict['PG'][0].avg_points+(position_player_dict['PG'][0].avg_points if position_player_dict['PG'][0].projected_avg_points==0 else position_player_dict['PG'][0].projected_avg_points))/2
                        total_projected_points+=(position_player_dict['PG'][1].avg_points+(position_player_dict['PG'][1].avg_points if position_player_dict['PG'][1].projected_avg_points==0 else position_player_dict['PG'][1].projected_avg_points))/2                     
                        players_ordered_list.remove(position_player_dict['PG'][0])
                        players_ordered_list.remove(position_player_dict['PG'][1])
                        position_only_players.append(position_player_dict['PG'][0])
                        position_only_players.append(position_player_dict['PG'][1])
                        players_only_position+=2
                        no_one_positon = False
                    if roster_spots['SG'] == None:
                        roster_spots['SG'] = position_player_dict['SG'][0]
                        roster_spots['G'] = position_player_dict['SG'][1]  
                        one_player_positions.append('SG')
                        one_player_positions.append('G')                            
                        total_projected_points+=(position_player_dict['SG'][0].avg_points+(position_player_dict['SG'][0].avg_points if position_player_dict['SG'][0].projected_avg_points==0 else position_player_dict['SG'][0].projected_avg_points))/2
                        total_projected_points+=(position_player_dict['SG'][1].avg_points+(position_player_dict['SG'][1].avg_points if position_player_dict['SG'][1].projected_avg_points==0 else position_player_dict['SG'][1].projected_avg_points))/2                     
                        players_ordered_list.remove(position_player_dict['SG'][0])
                        players_ordered_list.remove(position_player_dict['SG'][1])
                        position_only_players.append(position_player_dict['SG'][0])
                        position_only_players.append(position_player_dict['SG'][1])
                        players_only_position+=2
                        no_one_positon = False   

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1]) 
                            #position_player_dict[position2].remove(position_only_players[-1])

                        if position_only_players[-2] in players2:
                            players2.remove(position_only_players[-2])
                            #position_player_dict[position2].remove(position_only_players[-2])

                if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                    or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                    if roster_spots['SF'] == None:
                        roster_spots['SF'] = position_player_dict['SF'][0]
                        roster_spots['F'] = position_player_dict['SF'][1]  
                        one_player_positions.append('SF')
                        one_player_positions.append('F')                            
                        total_projected_points+=(position_player_dict['SF'][0].avg_points+(position_player_dict['SF'][0].avg_points if position_player_dict['SF'][0].projected_avg_points==0 else position_player_dict['SF'][0].projected_avg_points))/2
                        total_projected_points+=(position_player_dict['SF'][1].avg_points+(position_player_dict['SF'][1].avg_points if position_player_dict['SF'][1].projected_avg_points==0 else position_player_dict['SF'][1].projected_avg_points))/2                 
                        players_ordered_list.remove(position_player_dict['SF'][0])
                        players_ordered_list.remove(position_player_dict['SF'][1])
                        position_only_players.append(position_player_dict['SF'][0])
                        position_only_players.append(position_player_dict['SF'][1])
                        players_only_position+=2
                        no_one_positon = False
                    if roster_spots['PF'] == None:
                        roster_spots['PF'] = position_player_dict['PF'][0]
                        roster_spots['F'] = position_player_dict['PF'][1]  
                        one_player_positions.append('PF')
                        one_player_positions.append('F')                            
                        total_projected_points+=(position_player_dict['PF'][0].avg_points+(position_player_dict['PF'][0].avg_points if position_player_dict['PF'][0].projected_avg_points==0 else position_player_dict['PF'][0].projected_avg_points))/2
                        total_projected_points+=(position_player_dict['PF'][1].avg_points+(position_player_dict['PF'][1].avg_points if position_player_dict['PF'][1].projected_avg_points==0 else position_player_dict['PF'][1].projected_avg_points))/2                    
                        players_ordered_list.remove(position_player_dict['PF'][0])
                        players_ordered_list.remove(position_player_dict['PF'][1])
                        position_only_players.append(position_player_dict['PF'][0])
                        position_only_players.append(position_player_dict['PF'][1])
                        players_only_position+=2
                        no_one_positon = False   

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1]) 

                        if position_only_players[-2] in players2:
                            players2.remove(position_only_players[-2]) 

        num_empty_positions = len(empty_positions)   # Number of positions that will be empty
        """   if 'PG' in empty_positions and 'SG' in empty_positions:
            num_empty_positions+=1
        if 'SF' in empty_positions and 'PF' in empty_positions:
            num_empty_positions+=1
        #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
        #    print("aaaaaaaaa")
         #   num_empty_positions+=1
        #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
        #    print("bbbbbbbbbbbbbbbbbbb")
         #   num_empty_positions+=1
        """

        while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
            players_ordered_list_min = []

            if failed > 0:  # If first attempt fails
                print("FAILED " + str(failed))
                if failed == 1:
                    possible_iterations = list(itertools.permutations(players_ordered_list))

                if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                    if failed >= len(possible_iterations):
                        tried_every_combination = 1
                        day_projected_points[day] = max_points
                        print("Tried every Combo")
                        print(day_projected_points)
                        break
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If number of players playing is greater than starting spots
                    if combination_num == 0:  # If have not failed with all attmepts at original list
                        temp_list = players_ordered_list[:lineup_size]  # temporary list
                    if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                        players_ordered_list_min = possible_iterations[failed]
                    else:  # If have failed with all attempts at current list
                        combination_num += 1  # Will increase for each failure
                        temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                        players_ordered_list_min = temp_list
                        failed = 0  # Resetting fails

            else:
                players_ordered_list_min = players_ordered_list[:lineup_size]

            for player in players_ordered_list_min:
                eligible_slots = []

                for pos in position_order_list:
                    if pos in player.eligibleSlots:
                        eligible_slots.append(pos)

                for spot in eligible_slots:

                    if spot in roster_spots.keys() and spot != 'UT' and roster_spots[spot] == None:
                        roster_spots[spot] = player
                        players_list.append(player)
                        total_projected_points+=(player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2
                        current_num_players+=1
                        break
                    elif spot == 'UT' and len(roster_spots[spot]) < 3:
                        roster_spots[spot].append(player)
                        players_list.append(player)
                        total_projected_points+=(player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2
                        current_num_players+=1
                        break

            if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
                current_num_players = 0
                players_list = []
                if max_points < total_projected_points:   # If this iteration performed the best, make a note
                    max_points = total_projected_points

                total_projected_points = 0
                roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
                failed +=1
        if day not in day_projected_points.keys():
            day_projected_points[day] = total_projected_points

    return(day_projected_points, sum(day_projected_points.values()))

def daily_projected_points_old(week_num, day, team_player_dict, week_day_teams_df):
    pass
    print(day)
    position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    position_prime_number_dict = {'PG': 3, 'SG': 5, 'SF': 7, 'PF': 11, 'C': 13, 'G': 17, 'F': 19}
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']
    if day not in daily_players(week_num, team_player_dict, week_day_teams_df).keys():
        return 0

    players = daily_players(week_num, team_player_dict, week_day_teams_df)[day] 

    max_players = len(players)
    current_num_players = 0
    roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
    players_list = []
    total_projected_points = 0
    failed = 0
    combination_num = 0
    players_only_position = 0
    max_points = 0

    player_point_dict = {}
    for player in players:
        player_point_dict[player] = (player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2

    player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
    players_ordered_list = list(player_point_dict.keys())
    print(player_point_dict)
    position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
    for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
        for player in players_ordered_list:
            if position in player.eligibleSlots:
                position_player_dict[position].append(player)
    print(position_player_dict)
    position_only_players = []  # players who are the only player at a position
    one_player_positions = []
    no_one_positon = False  # we will loop again if we have a success
    empty_positions = []  # no players at position

    player_prime_num_dict = {}

    for player in players_ordered_list[:lineup_size]:
        eligible_slots = []

        player_prime_num_dict[player] = []

        for pos in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            if pos in player.eligibleSlots:
                player_prime_num_dict[player].append(position_prime_number_dict[pos])
    print(players_ordered_list[:lineup_size])
    if len(players_ordered_list[:lineup_size]) > 4:
        found_combo = False
        for combo in itertools.product(*player_prime_num_dict.values()):
            print(combo)
            player_position_score = functools.reduce(mul, combo)

            mult = 1
            for pos, l in position_player_dict.items():
                if len(l) == 0:
                    player_position_score*= position_prime_number_dict[pos]

            print(player_position_score)
            for val in [np.prod(list(itertools.combinations([3,5,7,11,13,17,19],len(players_ordered_list[:lineup_size])-3))[num]) for num in range(math.comb(7, len(players_ordered_list[:lineup_size])-3))]:

                if  player_position_score%(val) == 0:
                    found_combo = True
                    break
            if found_combo == True:
                break

        if found_combo == False:
            print("No combo Failure")
    print("Here")

    while no_one_positon == False:

        no_one_positon = True
        for position, players in position_player_dict.items():
            if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                empty_positions.append(position) 

            if len(players) == 1 and players[0] not in position_only_players:
                one_player_positions.append(position)
                roster_spots[position] = players[0]
                total_projected_points+=players[0].avg_points
                players_ordered_list.remove(players[0])
                position_only_players.append(players[0])
                players_only_position+=1
                no_one_positon = False

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1])
                        #position_player_dict[position2].remove(position_only_players[-1])

            if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                if roster_spots['PG'] == None:
                    roster_spots['PG'] = position_player_dict['PG'][0]
                    roster_spots['G'] = position_player_dict['PG'][1]  
                    one_player_positions.append('PG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['PG'][0].avg_points
                    total_projected_points+=position_player_dict['PG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PG'][0])
                    players_ordered_list.remove(position_player_dict['PG'][1])
                    position_only_players.append(position_player_dict['PG'][0])
                    position_only_players.append(position_player_dict['PG'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['SG'] == None:
                    roster_spots['SG'] = position_player_dict['SG'][0]
                    roster_spots['G'] = position_player_dict['SG'][1]  
                    one_player_positions.append('SG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['SG'][0].avg_points
                    total_projected_points+=position_player_dict['SG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SG'][0])
                    players_ordered_list.remove(position_player_dict['SG'][1])
                    position_only_players.append(position_player_dict['SG'][0])
                    position_only_players.append(position_player_dict['SG'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 
                        #position_player_dict[position2].remove(position_only_players[-1])

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2])
                        #position_player_dict[position2].remove(position_only_players[-2])

            if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                if roster_spots['SF'] == None:
                    roster_spots['SF'] = position_player_dict['SF'][0]
                    roster_spots['F'] = position_player_dict['SF'][1]  
                    one_player_positions.append('SF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['SF'][0].avg_points
                    total_projected_points+=position_player_dict['SF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SF'][0])
                    players_ordered_list.remove(position_player_dict['SF'][1])
                    position_only_players.append(position_player_dict['SF'][0])
                    position_only_players.append(position_player_dict['SF'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['PF'] == None:
                    roster_spots['PF'] = position_player_dict['PF'][0]
                    roster_spots['F'] = position_player_dict['PF'][1]  
                    one_player_positions.append('PF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['PF'][0].avg_points
                    total_projected_points+=position_player_dict['PF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PF'][0])
                    players_ordered_list.remove(position_player_dict['PF'][1])
                    position_only_players.append(position_player_dict['PF'][0])
                    position_only_players.append(position_player_dict['PF'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2]) 

    num_empty_positions = len(empty_positions)   # Number of positions that will be empty
    """   if 'PG' in empty_positions and 'SG' in empty_positions:
        num_empty_positions+=1
    if 'SF' in empty_positions and 'PF' in empty_positions:
        num_empty_positions+=1
    #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
    #    print("aaaaaaaaa")
     #   num_empty_positions+=1
    #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
    #    print("bbbbbbbbbbbbbbbbbbb")
     #   num_empty_positions+=1
    """
    print("current_num " + str(current_num_players))
    print("players_only_position " + str(players_only_position))
    print("num_empty_positions " + str(num_empty_positions))
    print("max player " + str(max_players))
    while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
        players_ordered_list_min = []

        if failed > 0:  # If first attempt fails
            print("FAILED " + str(failed))
            if failed == 1:
                #possible_iterations = list(itertools.permutations(players_ordered_list))

                for possible_iterations in list(itertools.permutations(players_ordered_list)):

                    players_list, roster_spots, current_num_players, total_projected_points = player_organizer(players_ordered_list_min, position_order_list, roster_sptos)

                    print("roster_spots")
                    print(roster_spots)
                    if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
                        current_num_players = 0
                        players_list = []
                        if max_points < total_projected_points:   # If this iteration performed the best, make a note
                            max_points = total_projected_points

                        total_projected_points = 0
                        roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
                        failed +=1
                        print("FAILED " + str(failed))

                    else:
                        return(total_projected_points)


                if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                    if failed >= len(possible_iterations):
                        tried_every_combination = 1
                        print("Tried every Combo")
                        break
                else:
                    if combination_num == 0:  # If have not failed with all attmepts at original list
                        temp_list = players_ordered_list[:lineup_size]  # temporary list

                    if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                        players_ordered_list_min = possible_iterations[failed]
                    else:  # If have failed with all attempts at current list
                        combination_num += 1  # Will increase for each failure
                        temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                        players_ordered_list_min = temp_list
                        failed = 0  # Resetting fails


            if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                if failed >= len(possible_iterations):
                    tried_every_combination = 1
                    print("Tried every Combo")
                    break
                players_ordered_list_min = possible_iterations[failed]
            else:  # If number of players playing is greater than starting spots
                if combination_num == 0:  # If have not failed with all attmepts at original list
                    temp_list = players_ordered_list[:lineup_size]  # temporary list
                if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If have failed with all attempts at current list
                    combination_num += 1  # Will increase for each failure
                    temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                    players_ordered_list_min = temp_list
                    failed = 0  # Resetting fails

        else:
            players_ordered_list_min = players_ordered_list[:lineup_size]

        players_list, roster_spots, current_num_players, total_projected_points = player_organizer(players_ordered_list_min, position_order_list, roster_spots)

        print("roster_spots")
        print(roster_spots)
        if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
            print(roster_spots)
            print(players_list)

            aaaaa = bubble_sorter()

            current_num_players = 0
            players_list = []
            if max_points < total_projected_points:   # If this iteration performed the best, make a note
                max_points = total_projected_points

            total_projected_points = 0
            roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
            failed +=1


    return(total_projected_points)

def player_organizer(list_players, position_order_list, team_roster_spots):
    
    projected_points = 0
    spots_filled = 0
    
    for player in list_players:
        eligible_slots = []

        for pos in position_order_list:
            if pos in player.eligibleSlots:
                eligible_slots.append(pos)

        for spot in eligible_slots:

            if spot in team_roster_spots.keys() and spot != 'UT' and team_roster_spots[spot] == None:
                team_roster_spots[spot] = player
                list_players.append(player)
                projected_points+=(player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2
                spots_filled+=1
                break
            elif spot == 'UT' and len(team_roster_spots[spot]) < 3:
                team_roster_spots[spot].append(player)
                list_players.append(player)
                projected_points+=(player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2
                spots_filled+=1
                break
                
    return list_players, team_roster_spots, spots_filled, projected_points

def pretty(d, indent=0):
    for key, value in d.items():
        if isinstance(value, dict):
            print(key)
            pretty(value, indent+1)
            
        #print('\t' * indent + str(key) + ": " + str(value) + '\n')
        else:
            print('\t' * indent + str(key) + ": " + str(value)) 

In [129]:
file_name = 'C:\\Users\\Jack\\Basketball Project\\NBA_Schedule_23_24.csv'
schedule_df = reading_data(file_name) # Retrieving Spreadsheet
league = League(league_id=1762898804, year=2024, espn_s2 = 'AEAkbmf6ntGyDFU%2BZ5MICm2WvwmwQ7SsLFomcaskbGz8VksoIPAE1D3MYbsTpcyiNBklJETozxCt1D0n3U7yat2RFOSQrW%2FqZdbWbsdxVWuRfOGKCfOfLrdDSYm2VsduCiWS0c5JLZ2zXzCg1UQqzReuFgIK8KLGyLj9%2FNe2IGgEgPqjwcqEp6PSBb6EssdzMJmuQMRgWtWIBE%2BRX1Ii%2FqLJTRTfLOchv6DK2%2FY%2BitmWtiY4Oa2n6gyF0Acup9V34nFNYB8IKmAZLpZqz3QFlTPw', swid='{A98C2FBC-429C-481F-8D11-8EDAE4526062}', debug=False)
free_agents = league.free_agents(size = 250)

In [130]:
def daily_projected_points(week_num, day, team_player_dict, week_day_teams_df):
    position_prime_number_dict = {'PG': 3, 'SG': 5, 'SF': 7, 'PF': 11, 'C': 13, 'G': 17, 'F': 19}
    position_lineup_spots_dict = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 3, 'F': 3}
    lineup_size = 10
    utility_spots = 3
    
    if day not in daily_players(week_num, team_player_dict, week_day_teams_df).keys():
        return 0
    else:
        players = daily_players(week_num, team_player_dict, week_day_teams_df)[day] 
        total_projected_points = 0
        only_position_projected_points = 0


        player_point_dict = {}
        for player in players:
            player_point_dict[player] = (player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2

        player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
        players_ordered_list = list(player_point_dict.keys())

        position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
        for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            for player in players_ordered_list:
                if position in player.eligibleSlots:
                    position_player_dict[position].append(player)
                    
        empty_positions = []
        only_player_at_pos = []

        for pos, l in position_player_dict.items():
            adjusted_list = [p for p in l if p not in only_player_at_pos]
            if pos in ('G', 'F') and pos not in empty_positions and len(adjusted_list) <= position_lineup_spots_dict[pos]:
                empty_positions.append(pos)
                for pl in adjusted_list:
                    player_point_dict.pop(pl) 
                    only_position_projected_points+=(pl.avg_points+(pl.avg_points if pl.projected_avg_points==0 else pl.projected_avg_points))/2
                    only_player_at_pos.append(pl)
                    players_ordered_list.remove(pl)
      
        keep_looping = True
        while keep_looping == True:
            keep_looping = False
            for pos, l in position_player_dict.items():   # Finding positions with no players so we can ignore them
                adjusted_list = [p for p in l if p not in only_player_at_pos]
                if len(adjusted_list) == 0 and pos not in empty_positions:
                    empty_positions.append(pos)
                
                elif len(adjusted_list) == 1 and adjusted_list[0] not in only_player_at_pos:
                    empty_positions.append(pos)
                    only_player_at_pos.append(adjusted_list[0])
                    players_ordered_list.remove(adjusted_list[0])
                    position_player_dict[pos] = []
                    player_point_dict.pop(adjusted_list[0])
                    only_position_projected_points+=(adjusted_list[0].avg_points+(adjusted_list[0].avg_points if adjusted_list[0].projected_avg_points==0 else adjusted_list[0].projected_avg_points))/2
                    keep_looping = True
                    
        if len(players_ordered_list) == 0:
            return only_position_projected_points
                
        prime_number_list = [v for k, v in position_prime_number_dict.items() if k not in empty_positions]    # Removing prme numbers associated with empty positions    
        ##print(only_player_at_pos)
        ##print(players_ordered_list)
        for attempted_player_count in range(min(lineup_size-len(empty_positions), len(players_ordered_list)), utility_spots+1, -1):
            ##print("players: " + str(players_ordered_list[:min(lineup_size-len(empty_positions), len(players_ordered_list))]))
            for player_combo in itertools.combinations(players_ordered_list, attempted_player_count):  # For all combinations of possible players 
                player_prime_num_dict = {}   # For each player, getting prime numbers associated with their positions

                for player in player_combo[:lineup_size]:
                    player_prime_num_dict[player] = []

                    for pos in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
                        if pos in player.eligibleSlots:
                            player_prime_num_dict[player].append(position_prime_number_dict[pos])
                ##print(player_prime_num_dict)
                found_combo = False 
                combo_mult_list = []
                for combo in itertools.product(*player_prime_num_dict.values()):  # Combinations of player prime numbers
                    ##print(combo)
                    ##print(combo_mult_list)
                    if math.prod(combo) in combo_mult_list:
                        #print('Already tried')
                        continue
                    combo_mult_list.append(sum(combo))
                    
                    player_position_score = functools.reduce(operator.mul, combo)   # Multiplying numbers in the given combination

                    for val in ({np.prod(list(itertools.combinations(prime_number_list,attempted_player_count-utility_spots))[num]) 
                                  for num in range(math.comb(len(prime_number_list), attempted_player_count-utility_spots))}):   # Possible combinations of products of prime numbers of availble position

                        if  player_position_score%val == 0:   # Success if the product of player prime numbers is divisible by the product of the position prime numbers
                            found_combo = True
                            ##print("Found Combo")            
                            total_projected_points = 0
                            for player in player_combo:   # After success, finding total projected points
                                total_projected_points += player_point_dict[player]
                            ##print("total_projected_points " + str(total_projected_points+ only_position_projected_points))    
                            return (total_projected_points + only_position_projected_points)

            print("No combo Failure")


        if len(players_ordered_list) <= utility_spots+1:
            for player in players_ordered_list:   # After success, finding total projected points
                total_projected_points += player_point_dict[player]
            ##print("total_projected_points " + str(total_projected_points+ only_position_projected_points)) 
            return (total_projected_points + only_position_projected_points)
    print('returning 0')    
    return 0

In [131]:
start_time = time.time()
week = 2
print("week " + str(week))
team = league.teams[5]
orig_roster = copy.copy(team.roster)
week_day_df = week_day_teams(schedule_df)
#orig_weekly_points_dict, orig_weekly_points = week_projected_points(week, week_day_df, team_player_dict_creator(orig_roster))



day_of_week = 0
prior_points = 0
original_points = 0

player_value_dict = {}
for day in week_day_df['Week ' + str(week)].keys(): 
    print(day)
    if day_of_week > 6:
        break
    if datetime.datetime.strptime(day, '%m/%d/%y').date() < datetime.datetime.today().date():
        day_of_week+=1
        continue
    #print("Making changes prior to " + day)
    #if day_of_week > 0:
     #   prior_points = 0
      #  for prev_day in week_day_df['Week ' + str(week)].keys():
      #      print(prev_day)
      #      if prev_day >= day:
      #          break
       #     prior_points+=daily_projected_points(week, prev_day, team_player_dict_creator(orig_roster), week_day_df)
    original_lineup_points = 0
    
    tt = 0
    for temp_day in week_day_df['Week ' + str(week)].keys():
        tt+=1
        if temp_day < day:   
            continue
        if tt > 6:
            break
        original_lineup_points+=daily_projected_points(week, temp_day, team_player_dict_creator(orig_roster), week_day_df)
        print("orig" + str(original_lineup_points))

    

    day_of_week+=1
    #print("Orginal Lineup points " + str(original_lineup_points))

    for player in team.roster:
        
        if (player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2 > 30 :
            continue

        else:
            if player not in player_value_dict.keys():
                player_value_dict[player] = {}
            for free_agent in free_agents:
                if free_agent.injuryStatus != 'ACTIVE' or (free_agent.avg_points+(free_agent.avg_points if free_agent.projected_avg_points==0 else free_agent.projected_avg_points))/2 < 18:
                    continue

                else:
                    #print("free agent " + free_agent.name)
                    #print('dropping player ' + player.name)
                    temp_roster = copy.copy(orig_roster)
                    temp_roster.remove(player)
                    temp_roster.append(free_agent)
                    temp_roster_dict = team_player_dict_creator(temp_roster)
                    new_lineup_points = 0
                    tt = 0
                    for temp_day in week_day_df['Week ' + str(week)].keys():
                        tt+=1
                        if temp_day < day:
                            continue
                        if tt > 6:
                            break
                        new_lineup_points+=daily_projected_points(week, temp_day, temp_roster_dict, week_day_df)

                    total_weekly_additional_points =  new_lineup_points - original_lineup_points   

                    if total_weekly_additional_points > 10:
                        if free_agent not in player_value_dict[player].keys():
                            player_value_dict[player][free_agent] = [day, total_weekly_additional_points]
                        elif total_weekly_additional_points >= player_value_dict[player][free_agent][1]:
                            player_value_dict[player][free_agent] = [day, total_weekly_additional_points]

                        player_value_dict[player] = {k: v for k, v in sorted(player_value_dict[player].items(), key=lambda item: item[1][1], reverse = True)} 
print('time: ' + str(time.time()-start_time))
final_player_value_dict = {}
for k, v in player_value_dict.items():
    final_player_value_dict[k] = dict(itertools.islice(v.items(), 5))

pretty(final_player_value_dict)

week 2
10/30/23
orig311.63000000000005
orig376.61000000000007
orig688.2400000000001
orig776.6400000000001
orig1003.0400000000001
orig1171.4750000000001
10/31/23
orig64.98
orig376.61000000000007
orig465.0100000000001
orig691.4100000000001
orig859.845
11/01/23
orig311.63000000000005
orig400.0300000000001
orig626.4300000000001
orig794.865
11/02/23
orig88.4
orig314.8
orig483.235
11/03/23
orig226.4
orig394.83500000000004
11/04/23
orig168.435
11/05/23
time: 1.3140912055969238
Player(Deandre Ayton)
	Player(Jordan Goodwin): ['10/31/23', 41.91500000000008]
	Player(Andrew Nembhard): ['10/30/23', 23.845000000000027]
	Player(De'Andre Hunter): ['11/04/23', 23.375]
	Player(Bogdan Bogdanovic): ['11/04/23', 23.20999999999998]
	Player(Alex Caruso): ['11/04/23', 22.25]
Player(Jalen Williams)
	Player(Jordan Goodwin): ['10/31/23', 38.10500000000002]
	Player(Andrew Nembhard): ['11/04/23', 23.620000000000005]
	Player(De'Andre Hunter): ['11/04/23', 23.375]
	Player(Bogdan Bogdanovic): ['11/04/23', 23.20999999